In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import nltk
# from nltk.corpus import stopwords
from gensim.parsing.preprocessing import remove_stopwords 
import json

# from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

import sys
sys.path.append("../AccommodationRecommender-Airbnb/")

from utils.text_similarity import get_embedings,get_similarity

import warnings

warnings.filterwarnings("ignore")


In [2]:
df = pd.read_csv("data/content_data/content_data.csv", encoding='utf-8')


In [3]:
# not_na_df["ItemDescriptionEmbedings"][0]

In [4]:
df_not_na_price  = df.dropna(subset=['ItemReview','Location','NumberOfGuest','NumberOfBedrooms', 'NumberOfBeds', 'NumberOfBaths',
                 'CurrentPrice'])

df_full_not_na = df.dropna()

In [5]:
# location in one list
df_not_na_price[['lower_country','lower_location','lower_city','lower_state']] = df_not_na_price[['Country','Location','City','State']].apply(lambda x: x.str.lower())
df_not_na_price['location_list'] = df_not_na_price['lower_location'].apply(lambda x: list(map(str.strip,x.split(","))))


df_full_not_na[['lower_country','lower_location','lower_city','lower_state']] = df_full_not_na[['Country','Location','City','State']].apply(lambda x: x.str.lower())
df_full_not_na['location_list'] = df_full_not_na['lower_location'].apply(lambda x: list(map(str.strip,x.split(","))))

    

In [6]:
df_not_na_price.to_csv("data/content_data/not_na_data_price.csv",index=False)
df_full_not_na.to_csv("data/content_data/full_not_na_data.csv",index=False)

In [7]:
# text embedings

In [8]:
df_full_not_na.columns

Index(['ID', 'ItemTitle', 'Section', 'ItemReview', 'ItemReviewBySections',
       'ItemRatings', 'HostLevel', 'Location', 'Country', 'State', 'City',
       'NumberOfGuest', 'NumberOfBedrooms', 'NumberOfBeds', 'NumberOfBaths',
       'CurrentPrice', 'PriceBefore', 'Currency', 'ItemUnit', 'PriceReview',
       'PriceRatings', 'ItemDescription', 'Amenities',
       'AmenitiesWithCategories', 'NotIncludedAmenity', 'Comments', 'URL',
       'lower_country', 'lower_location', 'lower_city', 'lower_state',
       'location_list'],
      dtype='object')

In [9]:
df_embedings = df_full_not_na[['ID','ItemDescription', 'Amenities', 'Comments']]

In [10]:
df_embedings

,ID,ItemDescription,Amenities,Comments
1,B5E327D844CA0024C61ADE0FA5606A3F_element_2602,"Freshly refurbished, it offers everything you ...","Bathtub, Hair dryer, Cleaning products, Hot wa...",['We had a lovely holiday and thought apartmen...
3,B5E327D844CA0024C61ADE0FA5606A3F_element_2604,A truly luxury class apartment set in beautifu...,"Beach view, Pool view, Garden view, Hair dryer...","['Very nice stay, place was comfortable and mo..."
13,B5E327D844CA0024C61ADE0FA5606A3F_element_2614,"COMPLEX WITH POOL, A LESS THAN 400 METERS FROM...","Pool view, Valley view, Garden view, Bathtub, ...",['We had a lovely week in Nerja!\nThe apartmen...
15,B5E327D844CA0024C61ADE0FA5606A3F_element_2616,"Comfortable and stylish, this traditional and ...","Courtyard view, Mountain view, Hair dryer, Sha...","['Absolutely Great!', 'This was the best Airbn..."
17,B5E327D844CA0024C61ADE0FA5606A3F_element_2618,"Our newly renovated 2 bed, 2 bath apartment is...","Garden view, Bay view, Hair dryer, Cleaning pr...","[""Helena's communications were great and her a..."
...,...,...,...,...
8855,68DB3E111AFB55A19D1E62E75E89B469_element_835,The City Oasis is one of five listings of the ...,"Courtyard view, City skyline view, Hair dryer,...","[""We thoroughly enjoyed our stay at Christine'..."
8858,FA13E440773D3D25A9290BCF42CBF87C_element_1898,Newly Renovated two-story villa DIRECTLY on Sa...,"Sea view, Ocean view, Beach view, Hair dryer, ...",['Lidiya was an incredible host! Very quick to...
8862,7CA9D0F980782E6657121EBA73AA5392_element_1376,The house is situated in the middle between G�...,"Hair dryer, Shampoo, Hot water, Essentials, Ha...",['Bart was a good host with lots of info on re...
8876,3FD56D3495BE98F0AC204DB14061F714_element_831,Take a break and unwind at this peaceful oasis...,"Garden view, Mountain view, Pool view, Hair dr...",['Very nice and secure place as described. Rah...


In [ ]:
df_embedings[['ItemDescriptionEmbedings', 'AmenitiesEmbedings', 'CommentsEmbedings']] = \
                df_embedings[['ItemDescription', 'Amenities','Comments']]\
                .apply(lambda x: [get_embedings(text,512,True) for text in x])


In [ ]:
emb_dict = df_embedings[['ID','CommentsEmbedings','ItemDescriptionEmbedings',"AmenitiesEmbedings"]].to_dict()

In [ ]:
for key in emb_dict:
    if key == "ID":
        continue
        
    for inner_key,value in emb_dict[key].items():
        emb_dict[key][inner_key] = value.tolist()


In [15]:
emb_dict


KeyboardInterrupt



In [17]:
with open("data/content_data/emdebings.json","w") as file:
    json.dump(emb_dict,file)

In [97]:
with open("emdbs.json","r") as file:
    ddict = json.load(file)

In [19]:
with open("data/content_data/emdebings.json",'r') as file:
    embedings = json.load(file)

for key in embedings:
    if key == "ID":
        continue
    
    for inner_key,value in embedings.items():
        embedings[key][inner_key] = np.array(value)


In [20]:
emd_df = pd.DataFrame(embedings)

In [ ]:
embedings

In [63]:
df_full_not_na.columns

Index(['ID', 'ItemTitle', 'Section', 'ItemReview', 'ItemReviewBySections',
       'ItemRatings', 'HostLevel', 'Location', 'Country', 'State', 'City',
       'NumberOfGuest', 'NumberOfBedrooms', 'NumberOfBeds', 'NumberOfBaths',
       'CurrentPrice', 'PriceBefore', 'Currency', 'ItemUnit', 'PriceReview',
       'PriceRatings', 'ItemDescription', 'Amenities',
       'AmenitiesWithCategories', 'NotIncludedAmenity', 'Comments',
       'ItemDescriptionEmbedings', 'AmenitiesEmbedings', 'CommentsEmbedings'],
      dtype='object')

In [58]:
# text embedings

In [35]:
# import sklearn
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

